<a href="https://colab.research.google.com/github/AnalyticsKnight/yemoonsaBigdata/blob/dev/CH4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4. A to Z 빅데이터 분석 실습  

## 2.실습 A to Z – Motor Trend Car Road Tests

### mtcars: Motor Trend Car Road Tests
- https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv

### (2) 사전준비

400

In [17]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("./mpg_X_test.csv")
X_train = pd.read_csv("./mpg_X_train.csv")
y_train = pd.read_csv("./mpg_y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'gender': pred}).to_csv('003000000.csv', index=False)

401

In [18]:
import pandas as pd
import numpy as np

402

In [19]:
# df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv")
X_test = pd.read_csv("./mpg_X_test.csv")
X_train = pd.read_csv("./mpg_X_train.csv")
y_train = pd.read_csv("./mpg_y_train.csv")

### (3) 데이터 전처리

403

In [20]:
print(X_train.info()) # 데이터 정보 및 결측치 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    275 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB
None


404

In [21]:
##### 결측치 처리하기
from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train[['horsepower']] = imputer.fit_transform( X_train[['horsepower']] )
X_test[['horsepower']] = imputer.fit_transform( X_test[['horsepower']] )

405

In [22]:
print(X_train.describe())

              mpg   cylinders  displacement  horsepower       weight  \
count  278.000000  278.000000    278.000000  278.000000   278.000000   
mean    23.616906    5.485612    193.501799  104.330909  2984.575540   
std      8.158168    1.724157    104.419891   37.577880   858.764682   
min      9.000000    3.000000     68.000000   48.000000  1613.000000   
25%     16.925000    4.000000     98.000000   75.000000  2206.250000   
50%     22.750000    4.000000    148.500000   95.000000  2866.500000   
75%     29.800000    8.000000    302.000000  129.000000  3641.750000   
max     46.600000    8.000000    455.000000  230.000000  4997.000000   

       acceleration  model_year  
count    278.000000  278.000000  
mean      15.581295   75.902878  
std        2.772807    3.577092  
min        8.000000   70.000000  
25%       13.800000   73.000000  
50%       15.300000   76.000000  
75%       17.000000   79.000000  
max       24.600000   82.000000  


406

In [23]:
# 불필요 변수
COL_DEL = ['name']
# 수치형 변수
COL_NUM = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']
# 범주형 변수 
COL_CAT = []
# 종속 변수
COL_Y = ['isUSA']

X_train = X_train.iloc[:, 1:]
X_test = X_test.iloc[:, 1:]

### (4) 데이터 모형 구축

407

In [24]:
##### 훈련데이터 분할
# y의 클래스 비율에 맞게 분할 stratify

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3)

408

In [25]:
##### 수치형 변수 - 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM]=scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM]=scaler.transform(X_val[COL_NUM])
X_test[COL_NUM]=scaler.transform(X_test[COL_NUM])

409

In [26]:
# KNN 분류
from sklearn.neighbors import KNeighborsClassifier
modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr.values.ravel())

# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth=10)
modelDT.fit(X_tr, y_tr)

DecisionTreeClassifier(max_depth=10)

410

In [27]:
y_val_pred = modelKNN.predict(X_val)

y_val_pred_probaKNN = modelKNN.predict_proba(X_val)
y_val_pred_probaDT = modelDT.predict_proba(X_val)

411

In [28]:
from sklearn.metrics import roc_auc_score

# roc_auc_score
scoreKNN = roc_auc_score(y_val, y_val_pred_probaKNN[:, 1])
scoreDT = roc_auc_score(y_val, y_val_pred_probaDT[:, 1])

print( scoreKNN, scoreDT )

0.9207175925925926 0.8628472222222223


412

In [29]:
best_model = None
best_score = 0

for i in range(2, 10):
    model = KNeighborsClassifier(n_neighbors=i, metric='euclidean')
    model.fit(X_tr, y_tr.values.ravel())
    y_val_pred_proba = model.predict_proba(X_val)
    score = roc_auc_score(y_val, y_val_pred_proba[:, 1])
    print(i,"개의 이웃 확인 : ", score)
    if best_score <= score:
        best_model = model

2 개의 이웃 확인 :  0.892650462962963
3 개의 이웃 확인 :  0.9010416666666667
4 개의 이웃 확인 :  0.9155092592592591
5 개의 이웃 확인 :  0.9207175925925926
6 개의 이웃 확인 :  0.9296875
7 개의 이웃 확인 :  0.9233217592592593
8 개의 이웃 확인 :  0.9244791666666666
9 개의 이웃 확인 :  0.9221643518518519


413

In [30]:
best_model.predict_proba(X_test)

array([[0.55555556, 0.44444444],
       [0.66666667, 0.33333333],
       [0.77777778, 0.22222222],
       [0.22222222, 0.77777778],
       [0.66666667, 0.33333333],
       [0.        , 1.        ],
       [0.55555556, 0.44444444],
       [0.55555556, 0.44444444],
       [0.66666667, 0.33333333],
       [0.11111111, 0.88888889],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.44444444, 0.55555556],
       [0.11111111, 0.88888889],
       [0.66666667, 0.33333333],
       [0.55555556, 0.44444444],
       [0.44444444, 0.55555556],
       [0.        , 1.        ],
       [0.77777778, 0.22222222],
       [0.22222222, 0.77777778],
       [0.33333333, 0.66666667],
       [0.        , 1.        ],
       [0.66666667, 0.33333333],
       [0.44444444, 0.55555556],
       [0.66666667, 0.33333333],
       [0.11111111, 0.88888889],
       [0.66666667, 0.33333333],
       [0.        , 1.        ],
       [0.77777778, 0.22222222],
       [0.        , 1.        ],
       [0.

414

In [31]:
pred = best_model.predict_proba(X_test)[:,1]
print(pred)

[0.44444444 0.33333333 0.22222222 0.77777778 0.33333333 1.
 0.44444444 0.44444444 0.33333333 0.88888889 1.         1.
 0.55555556 0.88888889 0.33333333 0.44444444 0.55555556 1.
 0.22222222 0.77777778 0.66666667 1.         0.33333333 0.55555556
 0.33333333 0.88888889 0.33333333 1.         0.22222222 1.
 0.66666667 0.77777778 0.44444444 0.11111111 1.         0.44444444
 1.         1.         1.         1.         1.         1.
 1.         0.55555556 1.         0.55555556 0.66666667 1.
 0.44444444 0.77777778 0.77777778 0.33333333 0.44444444 0.22222222
 1.         0.33333333 1.         0.88888889 0.22222222 0.55555556
 1.         1.         0.33333333 0.55555556 0.77777778 0.11111111
 0.66666667 0.44444444 0.55555556 0.44444444 1.         0.77777778
 1.         0.         1.         1.         1.         1.
 0.55555556 1.         0.88888889 0.22222222 0.77777778 1.
 1.         1.         0.33333333 0.11111111 1.         0.
 1.         1.         0.44444444 1.         0.44444444 0.44444444


415

In [32]:
pd.DataFrame({'isUSA': pred}).to_csv('003000000.csv', index=False)